In [1]:
# Import 
import pandas as pd 
import numpy as np
import mne
import matplotlib.pyplot as plt 

In [8]:
# Import dataset - DREAM 
file = "../dataset/DREAM/Zhang & Wamsley 2019/Data/PSG/subject026_Morning.edf"
data = mne.io.read_raw_edf(file)
dream_df = data.get_data()
# Metadata 
info = data.info
channels = data.ch_names
raw_df = pd.DataFrame(dream_df)
raw_df.index = channels

Extracting EDF parameters from c:\Users\PC\Documents\Personal Projects\AI_Honor_track\project\TUe-Honors-MedAI\dataset\DREAM\Zhang & Wamsley 2019\Data\PSG\subject026_Morning.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
